In [1]:
import pickle

In [2]:
import pickle
import re
import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import nltk
from nltk.corpus import stopwords
from snowballstemmer import TurkishStemmer


In [3]:
with open('articles_2700.pickle', 'rb') as f:
    queries = pickle.load(f)

In [4]:
queries[0]

{'ArticleTitle': ['Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller'],
 'Author': ['ÖZTAN, Güven Gürkan\n                        ',
  'BEZCİ, Egemen B.\n                        '],
 'Publisher': ['Mülkiyeliler Birliği Genel Merkezi',
  'Mülkiye Alumni Association'],
 'Date': ['2015-04-10'],
 'DergiParkLink': ['https://dergipark.org.tr/tr/pub/mulkiye/issue/277/1251',
  None],
 'PDFLink': ['https://dergipark.org.tr/tr/download/article-file/1073'],
 'References': ['Agamben G (1998). Homo Sacer: Sovereign Power and Bare Life. California: Stanford University Press.',
  'Agamben G (2005). State of Exception. Chicago: University of Chicago Press.',
  'Aydınlı E, Özcan N A ve Akyaz D (2006). The Turkish Military’s March Toward Europe. Foreign Affairs, 85, 77-90.',
  'Benjamin W (2010). Şiddetin Eleştirisi Üzerine. İçinde: çev. E Göztepe, A Çelebi (der), Şiddetin Eleştirisi Üzerine, İstanbul: Metis, 19-43.',
  'Bezci E B (2014). Intelligence Reform and Threat Perception in Turkey. J

In [5]:
len(queries[0]["References"])

55

In [6]:
title = queries[0]["ArticleTitle"]
title = ''.join(title).replace('\n', ' ').strip()

author = queries[0]["Author"]
author = ''.join(author).replace('\n', ' ').strip()

In [7]:
year = queries[0]["Date"]

In [8]:
year = ''.join(year)[:4]

In [9]:
year

'2015'

In [10]:
import re
import PyPDF2

In [11]:
def pdf_to_text(pdf_file_path):
    text = ""
    with open(pdf_file_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    text = text.replace('\n', ' ')
    return text

In [12]:
text = pdf_to_text("000.pdf")

In [13]:
text

'159 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.Türkiye’de Olağanüstü Hal:   Devlet Aklı, Askerler ve Siviller Güven Gürkan Öztan , İstanbul Üniversitesi Siyasal Bilgiler Fakültesi,  e-posta: guven.oztan@istanbul.edu.tr. Egemen B. Bezci, Nottingham Üniversitesi Tarih Bölümü,   e-posta: ahxebbe@nottingham.ac.uk. Özet 12 Eylül 1980 askeri darbesi Türkiye’de sosyal, ekonomik ve siyasal alanda geniş çaplı  değişikliklere yol açmıştır. Aynı zamanda darbe yönetimi, 1982 yılında yürürlü ğe giren  ve günümüzde de geçerliliğini sürdüren Anayasa’yı hazırlamıştır.  1982 anayasanın  önemli özelliklerinden biri olağanüstü hal kavramını Türkiye’nin siyasi arenasına  taşıması olmuştur. 1982 Anayasası ile birlikte anayasal düzene dâhil olan olağanüstü  hal, mevcut sıkıyönetim uygulamalarının yerine geçerek 1 Mart 1984 tarihinden  başlayarak uygulamadan kalktığı 30 Kasım 2002 tarihine kadar varlığını sürdürmüştür.  Bu ç

In [14]:
pattern_to_remove =  r'\b(\d+)\s+([\w\s]+)\((\d{4})\)\.\s*([^\.]+)\.\s*([^,]+),\s*(\d+)\s*\((\d+)\),\s*(\d+)-(\d+)\.\b'
matches = re.finditer(pattern_to_remove, text)

# Initialize a count variable
count = 0

# Iterate through the matches and count them
for match in matches:
    count += 1
    matched_sentence = match.group()
    print(matched_sentence)
    print("\n--------------\n")

# Print the total count of matching sentences
print(f"Total matches found: {count}")

159 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

160 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

161 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

162 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

163 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

164 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye Dergisi , 39(1), 159-186.

--------------

165 Öztan G G ve Bezci E B (2015). Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller.  Mülkiye 

In [15]:
text = re.sub(pattern_to_remove, '', text)
text

'Türkiye’de Olağanüstü Hal:   Devlet Aklı, Askerler ve Siviller Güven Gürkan Öztan , İstanbul Üniversitesi Siyasal Bilgiler Fakültesi,  e-posta: guven.oztan@istanbul.edu.tr. Egemen B. Bezci, Nottingham Üniversitesi Tarih Bölümü,   e-posta: ahxebbe@nottingham.ac.uk. Özet 12 Eylül 1980 askeri darbesi Türkiye’de sosyal, ekonomik ve siyasal alanda geniş çaplı  değişikliklere yol açmıştır. Aynı zamanda darbe yönetimi, 1982 yılında yürürlü ğe giren  ve günümüzde de geçerliliğini sürdüren Anayasa’yı hazırlamıştır.  1982 anayasanın  önemli özelliklerinden biri olağanüstü hal kavramını Türkiye’nin siyasi arenasına  taşıması olmuştur. 1982 Anayasası ile birlikte anayasal düzene dâhil olan olağanüstü  hal, mevcut sıkıyönetim uygulamalarının yerine geçerek 1 Mart 1984 tarihinden  başlayarak uygulamadan kalktığı 30 Kasım 2002 tarihine kadar varlığını sürdürmüştür.  Bu çalışma, olağanüstü hal uygulamasının devlet aklının bir tezahürü olduğunu iddia  etmektedir. Tehdit algılamalarına göre şekillenen 

In [16]:
def split_article(article_text):
    reference_pattern = r"Kaynakça|References|Bibliography|Citations|Sonnotlar"
    reference_match = re.search(reference_pattern, article_text, re.IGNORECASE)

    if reference_match:
        main_text = article_text[:reference_match.start()] #extract main text from references
        reference_section = article_text[reference_match.start():]

        return main_text, reference_section
    else:
        return article_text, None

text, references = split_article(text)

In [17]:
def window(text, loc, target_str, left_window_size=150, right_window_size=100):
    left_threshold = max(loc - left_window_size, 0)
    right_threshold = min(loc + right_window_size, len(text))

    return text[left_threshold:right_threshold]


def find_references_with_context(text, title, author, year):
    pattern = re.compile(r'\(.*?\d{4}.*?\)')
    references_with_context = []

    for match in pattern.finditer(text):
        if match.group().lower() != title.lower() and author not in match.group() and str(year) not in match.group():
            match_text = match.group()
            start_loc = text.find(match_text)
            context = window(text, start_loc, match_text, 150, 100)
            references_with_context.append(context)

    return references_with_context

In [18]:
sentences_with_references=find_references_with_context(text, title, author, year)

In [19]:
sentences_with_references

['y in Turkey and derive  the necessary empirical argument in the theoretical discussion, this article analyses  minutes of the National Security Board (Milli Güvenlik Konseyi) and Consultation  Assembly (Danışma Meclisi) during the relevant discussion',
 'incelenmesi bağlamında aynı saptamayı yapmak zordur; olağanüstü  hal rejimini odağına yerleştiren araştırmaların sayısı ve kapsamı görece sınırlıdır  (Gemalmaz, 1991; Üskül, 1988; Duran, 1995).  1980 ile 1983 yılları arasındaki  süreçte ülkeyi doğrud',
 ' Üskül, 1988; Duran, 1995).  1980 ile 1983 yılları arasındaki  süreçte ülkeyi doğrudan yöneten askeri rejim, orta vadedeki çabalarını devlet  aklının (raison d’état) çerçevesini çizdiği yeni bir anayasa hazırlatmak üzerine yoğunlaştırmıştır. Yeni ana',
 'ile silahlı mücadelesinde devlet otoritesini  yeniden tesis etmek adına olağanüstü hal rejiminden yararlandığını ve askerlerin  Milli Güvenlik Kurulu (MGK) üzerinden olağanüstü hale karar veren esas yetkili  merci ve siyasal özne old

In [20]:
for i in sentences_with_references:
    print(i)
    print("--------")

y in Turkey and derive  the necessary empirical argument in the theoretical discussion, this article analyses  minutes of the National Security Board (Milli Güvenlik Konseyi) and Consultation  Assembly (Danışma Meclisi) during the relevant discussion
--------
incelenmesi bağlamında aynı saptamayı yapmak zordur; olağanüstü  hal rejimini odağına yerleştiren araştırmaların sayısı ve kapsamı görece sınırlıdır  (Gemalmaz, 1991; Üskül, 1988; Duran, 1995).  1980 ile 1983 yılları arasındaki  süreçte ülkeyi doğrud
--------
 Üskül, 1988; Duran, 1995).  1980 ile 1983 yılları arasındaki  süreçte ülkeyi doğrudan yöneten askeri rejim, orta vadedeki çabalarını devlet  aklının (raison d’état) çerçevesini çizdiği yeni bir anayasa hazırlatmak üzerine yoğunlaştırmıştır. Yeni ana
--------
ile silahlı mücadelesinde devlet otoritesini  yeniden tesis etmek adına olağanüstü hal rejiminden yararlandığını ve askerlerin  Milli Güvenlik Kurulu (MGK) üzerinden olağanüstü hale karar veren esas yetkili  merci ve siy

In [21]:
len(sentences_with_references)

53

In [22]:
sentences_with_references = list(set(sentences_with_references))

In [23]:
references = queries[0]["References"]

In [24]:
len(references)

55

In [25]:
references

['Agamben G (1998). Homo Sacer: Sovereign Power and Bare Life. California: Stanford University Press.',
 'Agamben G (2005). State of Exception. Chicago: University of Chicago Press.',
 'Aydınlı E, Özcan N A ve Akyaz D (2006). The Turkish Military’s March Toward Europe. Foreign Affairs, 85, 77-90.',
 'Benjamin W (2010). Şiddetin Eleştirisi Üzerine. İçinde: çev. E Göztepe, A Çelebi (der), Şiddetin Eleştirisi Üzerine, İstanbul: Metis, 19-43.',
 'Bezci E B (2014). Intelligence Reform and Threat Perception in Turkey. Journal of Mediterranean and Balkan Intelligence, 3, 25-42.',
 'Bilgiç T Ü (2009). The Military and Europenization Reforms in Turkey. Middle Eastern Studies, 45, 803-824.',
 'Bozaslan H (2014). “Neden Silahlı Mücadele?”: Türkiye Kürdistanı’nda Şiddeti Anlamak. İçinde: Çeğin G ve Şirin İ (der), Türkiye’de Siyasal Şiddetin Boyutları. İstanbul: İletişim Yayınları, 149-164.',
 'Brown J (1989). The Military and Society: The Turkish Case. Middle Eastern Studies, 25, 387-404.',
 'Cizr

In [26]:
sentences_with_references

['Anayasanın kabulü ve 1983 seçimleriyle birlikte sivil siyasete  geçilirken MGK ilk kez 10 Kasım 1983 tarihinde sıkıyönetimi uzatma kararı  almaktadır (Üskül, 1988: 103). Silahlı Kuvvetler, olağanüstü hal koşulları altında  belirlediği normların sivil',
 'ci  grupların himaye ve teşvik ettiklerini, benzer olaylara son verilmesi için  Barzani kuvvetlerinin de te’dip edilmesinin” gerektiğini belirtmiştir (Milliyet,  25/08/1984: 6). 1990’larda Olağanüstü Hal ve TSK’nın Değişen Yöntemleri 1980’lerin ikinc',
 'ebilir ki, ordu, önce entellejensiyayı tek başına teşkil eden  bir unsur; daha sonraları da onun kayda değer müessiriyetleri olan bir unsur  olmuştur (1961: 39-54). Müesses nizam ile kendi varlığı arasında ontolojik bir ilişki kuran TSK, 12 Eylül  19',
 'ecektir. Bu durumda, esasen kaynağını Anayasanın  özünden almayan böyle bir Kanunun, antidemokratik kurallarını uygulayacak  bir hükümetin, kendisini (...hukuk dışı tutum ve davranışlarıyla meşruluğunu  kaybetmiş bir iktidar...) olma

In [27]:
def extract_citations(window):
    pattern = re.compile(r'\(.*?\d{4}.*?\)')
    return pattern.findall(window)

def match_reference(citation, references):
    # Extract year and try to find a part of the author's name
    match = re.search(r'(\d{4})', citation)
    if match:
        year = match.group(1)
        # Split citation and look for potential author name parts
        for part in re.split(r'\W+', citation):
            if part and len(part) > 2 and part.lower() != year:  # Exclude very short parts and the year
                for ref in references:
                    if year in ref and part in ref:
                        return ref
    return "No matching reference found"

In [28]:
for window in sentences_with_references:
    citations = extract_citations(window)
    for citation in citations:
        matched_reference = match_reference(citation, references)
        print(f"Window: {window}")
        print(f"Citation: {citation}")
        print(f"Matched Reference: {matched_reference}\n")

Window: Anayasanın kabulü ve 1983 seçimleriyle birlikte sivil siyasete  geçilirken MGK ilk kez 10 Kasım 1983 tarihinde sıkıyönetimi uzatma kararı  almaktadır (Üskül, 1988: 103). Silahlı Kuvvetler, olağanüstü hal koşulları altında  belirlediği normların sivil
Citation: (Üskül, 1988: 103)
Matched Reference: Üskül Z (1988). Türkiye’de Sıkıyönetim Uygulamaları Üzerine Notlar. Toplum ve Bilim,42, 85-104.

Window: ci  grupların himaye ve teşvik ettiklerini, benzer olaylara son verilmesi için  Barzani kuvvetlerinin de te’dip edilmesinin” gerektiğini belirtmiştir (Milliyet,  25/08/1984: 6). 1990’larda Olağanüstü Hal ve TSK’nın Değişen Yöntemleri 1980’lerin ikinc
Citation: (Milliyet,  25/08/1984: 6)
Matched Reference: Milliyet (25/08/1984).

Window: ebilir ki, ordu, önce entellejensiyayı tek başına teşkil eden  bir unsur; daha sonraları da onun kayda değer müessiriyetleri olan bir unsur  olmuştur (1961: 39-54). Müesses nizam ile kendi varlığı arasında ontolojik bir ilişki kuran TSK, 12 Eylül  1

In [29]:
citations_2 = []

for window in sentences_with_references:
    citations = extract_citations(window)
    for citation in citations:
        citations_2.append(citation)

In [30]:
def extract_citations(window):
    pattern = re.compile(r'\(.*?\d{4}.*?\)')
    return pattern.findall(window)

In [31]:
def calculate_cosine_similarity_tfidf(citations, references):
    # Combine both lists of strings
    all_strings = citations + references

    # Create a TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Transform the strings into TF-IDF vectors
    tfidf_matrix = vectorizer.fit_transform(all_strings)

    # Calculate cosine similarity between each pair of strings
    similarity_matrix = cosine_similarity(tfidf_matrix)

    # Iterate through citations and find the best match in references
    for i, citation in enumerate(citations):
        # Find the index of the most similar reference
        best_match_index = similarity_matrix[i, len(citations):].argmax()

        # Get the corresponding reference string
        best_match_reference = references[best_match_index]

        # Get the cosine similarity score
        similarity_score = similarity_matrix[i, len(citations) + best_match_index]

        # Print the results
        if similarity_score > 0.25:
            print(f"Citation: {citation}")
            print(f"Reference: {best_match_reference}")
            print(f"Cosine Similarity Score: {similarity_score:.4f}")
            print("-" * 30)

calculate_cosine_similarity_tfidf(citations_2, references)

Citation: (Üskül, 1988: 103)
Reference: Üskül Z (1988). Türkiye’de Sıkıyönetim Uygulamaları Üzerine Notlar. Toplum ve Bilim,42, 85-104.
Cosine Similarity Score: 0.2712
------------------------------
Citation: (Milliyet,  25/08/1984: 6)
Reference: Milliyet (25/08/1984).
Cosine Similarity Score: 1.0000
------------------------------
Citation: (1961: 39-54)
Reference: Savcı B (1961). Siyasi İlimler ve Hukuk II: Türkiye’de Devlet Hayatında Askeri Mahiyetin ve Tesirin Seyrine Bir Bakış. Ankara Üniversitesi Siyasal Bilgiler Fakültesi Dergisi, 16, 39-54.
Cosine Similarity Score: 0.3337
------------------------------
Citation: (Danışma Meclisi, 1983: 4)
Reference: Danışma Meclisi (1983). Olağanüstü Hal Kanun Tasarısı. Danışma Meclisi Tutanak Dergisi. Birleşim: 153, Oturum: 1, 29.08.1983
Cosine Similarity Score: 0.6482
------------------------------
Citation: (Aydınlı, Özcan ve Akyaz, 2006: 78)
Reference: Aydınlı E, Özcan N A ve Akyaz D (2006). The Turkish Military’s March Toward Europe. Foreig

In [32]:
citations_2

['(Üskül, 1988: 103)',
 '(Milliyet,  25/08/1984: 6)',
 '(1961: 39-54)',
 '(2005: 10- 12)',
 '(Danışma Meclisi, 1983: 4)',
 '(2014: 158)',
 '(Aydınlı, Özcan ve Akyaz, 2006: 78)',
 '(2007: 15)',
 '(1984: 2)',
 '(Danışma Meclisi, 1983:  33)',
 '(İnsel, 1997:  15-19)',
 '(2000: 116)',
 '(Bozdemir,  1982: 163)',
 '(Özcan, 2007: 43)',
 '(Yüksekova Haber, 14/08/2011)',
 '(Milliyet, 25/01/1997: 7)',
 '(Gemalmaz, 1991; Üskül, 1988; Duran, 1995)',
 '(07/10/1984: 7)',
 '(Öztan, 2014: 192)',
 '(2010: 409-410)',
 '(Bezci, 2014: 29-34)',
 '(1999: 31)',
 '(Cizre, 1999: 44)',
 '(Agamben, 1998: 165)',
 '(Danışma Meclisi, 1983: 3)',
 '(2009: 224)',
 '(2004: 143)',
 '(2005: 647)',
 '(2014: 221)',
 '(bu tür kararların kaldırılabilmesi için yeniden sıkıyönetim  ilanını istemek gerekiyor!) (1988: 89)',
 '(Milliyet, 03/10/1984: 6)',
 '(Danışma Meclisi, 1983: 6)',
 '(2005: 10- 12)',
 '(2006: 194-204)',
 '(2000: 95)',
 '(Cizre, 1993: 277)',
 '(1984)',
 '(Milliyet, 30/07/1996: 16)',
 '(Milliyet, 27/06/1995: 15)

In [33]:
len(references)

55

In [34]:
def extract_titles(references):
    titles = []
    for ref in references:
        # Use regex to find the pattern that matches the title
        match = re.search(r'\)\. (.+?)\.', ref)
        if match:
            titles.append(match.group(1).lower())
    return titles

titles = extract_titles(references)
for title in titles:
    print(title)


homo sacer: sovereign power and bare life
state of exception
the turkish military’s march toward europe
şiddetin eleştirisi üzerine
intelligence reform and threat perception in turkey
the military and europenization reforms in turkey
“neden silahlı mücadele?”: türkiye kürdistanı’nda şiddeti anlamak
the military and society: the turkish case
ap-ordu i̇lişkileri: bir i̇kilemin anatomisi
muktedirlerin siyaseti: merkez sağ-ordu-i̇slamcılık
democratic transition in turkey, 1980–83: the impact of european diplomacy
olağanüstü hal kanun tasarısı
olağanüstü hal kanun tasarısı
lessons of military regimes and democracy: the turkish case in a comparative perspective
bunalım kararnameleri
devlet ve politik şiddet: latin amerika ve türkiye’de gerilla hareketleri
security, territory, population: lectures at the college de france 1977-1978
historical roots of martial law within the turkish legal system: perspectives and text
güneydoğu anadolu’da devam etmekte olan bölücü hareketin gelecekteki muhteme

In [92]:
references

['Agamben G (1998). Homo Sacer: Sovereign Power and Bare Life. California: Stanford University Press.',
 'Agamben G (2005). State of Exception. Chicago: University of Chicago Press.',
 'Aydınlı E, Özcan N A ve Akyaz D (2006). The Turkish Military’s March Toward Europe. Foreign Affairs, 85, 77-90.',
 'Benjamin W (2010). Şiddetin Eleştirisi Üzerine. İçinde: çev. E Göztepe, A Çelebi (der), Şiddetin Eleştirisi Üzerine, İstanbul: Metis, 19-43.',
 'Bezci E B (2014). Intelligence Reform and Threat Perception in Turkey. Journal of Mediterranean and Balkan Intelligence, 3, 25-42.',
 'Bilgiç T Ü (2009). The Military and Europenization Reforms in Turkey. Middle Eastern Studies, 45, 803-824.',
 'Bozaslan H (2014). “Neden Silahlı Mücadele?”: Türkiye Kürdistanı’nda Şiddeti Anlamak. İçinde: Çeğin G ve Şirin İ (der), Türkiye’de Siyasal Şiddetin Boyutları. İstanbul: İletişim Yayınları, 149-164.',
 'Brown J (1989). The Military and Society: The Turkish Case. Middle Eastern Studies, 25, 387-404.',
 'Cizr

In [35]:
print(len(titles))

39


In [36]:
"Türkiye’de Olağanüstü Hal: Devlet Aklı, Askerler ve Siviller" in queries[0]['ArticleTitle']

True

In [48]:
articles_with_lowercase_titles = [
    {**article, 'ArticleTitle': [title.lower() for title in article.get('ArticleTitle', [])]}
    for article in queries
]

In [84]:
target_title = "Alkol-Madde Kullanıcılarının Bağımlılık Şiddeti İle Aile Üyelerinin Eş Bağımlılık Düzeyleri Arasındaki İlişki".lower()

In [87]:
matching_queries = [query for query in articles_with_lowercase_titles if 'ArticleTitle' in query and target_title in query['ArticleTitle']]

In [88]:
matching_queries

[{'ArticleTitle': ['alkol-madde kullanıcılarının bağımlılık şiddeti i̇le aile üyelerinin eş bağımlılık düzeyleri arasındaki i̇lişki',
   'relationship between addiction severity of alcohol-substance users and co-dependency levels of family members'],
  'Author': ['KAPLAN, Özgen\n                        ',
   'ÖZBARAN, Burcu\n                        '],
  'Publisher': ['Galenos Yayınevi', 'Galenos Yayınevi'],
  'Date': ['2023-07-03'],
  'DergiParkLink': ['https://dergipark.org.tr/tr/pub/bagimli/issue/80424/1322007',
   '10.51982/bagimli.1322007'],
  'PDFLink': ['https://dergipark.org.tr/tr/download/article-file/3240770'],
  'References': ['Tot Ş, Yazıcı K, Yazıcı A, et al. Psychosocial correlates of subs-tance use among adolescents in Mersin, Turkey. Public Health 2004; 118(8): 588-593',
   'Saunders EC, McLeman BM, McGovern MP, et al. The influence of family and social problems on treatment outcomes of persons with cooccurring substance use disorders and PTSD. J Subst Use 2016; 21(3): 

In [89]:
temp_ref = matching_queries[0]["References"]

In [91]:
temp_ref

['Tot Ş, Yazıcı K, Yazıcı A, et al. Psychosocial correlates of subs-tance use among adolescents in Mersin, Turkey. Public Health 2004; 118(8): 588-593',
 'Saunders EC, McLeman BM, McGovern MP, et al. The influence of family and social problems on treatment outcomes of persons with cooccurring substance use disorders and PTSD. J Subst Use 2016; 21(3): 237-243',
 'Ögel K, Çelikay H, Başabak A. Aile bağımlılık profil indeksinin geliştirilmesi, geçerlik ve güvenirliği. Klinik Psikiyatri Dergisi 2017; 20(4): 255-262.',
 'Aka P, Özok Hİ. Eş Bağımlılık. Tanhan F (editör). Psikolojik Danışmada Güncel Konular. Ankara, 2022; 39-45.',
 'Lancer D. Conquering Shame and Codependency: 8 Steps To Freeing The True You. Minnesota: Hazelden Publishing, 2014; 1-23.',
 'Ançel G. Karşılıklı bağımlılık kavramı: hemşirelikle ilişkisi ve karşılıklı bağımlılığı belirleme araçları. Hemşirelikte Araştırma Geliştirme Dergisi 2012; 14(1): 70-78.',
 'Zerwekh J, Michaels B. Co-dependency: assessment and recovery. Nur

In [98]:
titles = []
for ref in temp_ref:
    match = re.search(r'\. (.+?)\. [A-Z][a-z]+', ref)
    if match:
        titles.append(match.group(1).lower())

In [99]:
titles

['psychosocial correlates of subs-tance use among adolescents in mersin, turkey',
 'aile bağımlılık profil indeksinin geliştirilmesi, geçerlik ve güvenirliği',
 'eş bağımlılık',
 'conquering shame and codependency: 8 steps to freeing the true you',
 'karşılıklı bağımlılık kavramı: hemşirelikle ilişkisi ve karşılıklı bağımlılığı belirleme araçları',
 'co-dependency: assessment and recovery',
 'et al, the questionnaire of family functioning: a preliminary validation of a standardized instrument to evaluate psychoeducational family treatments',
 'co-dependency assessment inventory: a preliminary research tool',
 'a critique of the codependency concept considering the best interests of the child',
 'co‐dependency: a critical review',
 'choice making: for co-dependents, adult children and spirituality seekers',
 'codependency: an empirical study from a systemic perspective',
 'family stressors as predictors of codependency',
 'codependency explored: a social movement in search of definition

In [108]:
matched = [query for query in articles_with_lowercase_titles if 'ArticleTitle' in query and any(ref in query['ArticleTitle'] for ref in titles)]

In [109]:
matched

[{'ArticleTitle': ['bağımlılıkta eş bağımlılık faktörü (beşf) ölçeği’nin geçerlik ve güvenilirlik çalışması',
   'validity and reliability of co-dependency in substance use disorder scale (codsuds)'],
  'Author': ['ŞİMŞEK, Melike\n                        ',
   'ÖNCÜ, Fatihcan\n                        ',
   'KABİL, Melekcan\n                        '],
  'Publisher': ['Lut TAMAM', 'Lut TAMAM'],
  'Date': ['2020-09-27'],
  'DergiParkLink': ['https://dergipark.org.tr/tr/pub/pgy/issue/48355/800752',
   '10.18863/pgy.800752'],
  'PDFLink': ['https://dergipark.org.tr/tr/download/article-file/1312168']},
 {'ArticleTitle': ['üniversite öğrencilerinde madde kullanımı, şiddet ve bazı psikolojik özellikler',
   'üniversite öğrencilerinde madde kullanımı, şiddet ve bazı psikolojik  özellikler'],
  'Author': ['TURHAN, Ebru\n                        ',
   'İNANDI, Tacettin\n                        ',
   'ÖZER, Cahit\n                        ',
   'AKOĞLU, Sabahat\n                        '],
  'Publi